# data analysis and plotting results

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as pjoin
import seaborn as sns
%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':'tight'}

from robustness_eval import get_results, get_distortions_from_file

## general definitions

In [ ]:
result_dir = "../raw-data/"
figures_dir = "../figures/results/"

In [ ]:
sns.set()
sns.set_style('ticks')
sns.set_context('paper',rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":8})

In [ ]:
# Colour scheme

col_clean = tuple(x/255.0 for x in (50, 65, 75))
col_stylized = tuple(x/255.0 for x in (180, 160, 105))
col_combined = tuple(x/255.0 for x in (165, 30, 55))

In [ ]:
# plotting parameters

combined_plot_height = 7.0
combined_plot_width = 6.0

markersize = 16
fontsize = 24
linewidth = 2.5
labelsize = 18
legend_fontsize=18
axis_start = -0.2
axis_end = 5.3


In [ ]:
def plot_individual_results(data_A, data_B, data_C, plot_name, metric_name,
                            ylim, legend_loc=1, markersize=12, linewidth=2.0,
                            fontsize=15, legend_fontsize=11.3, labelsize=12.0,
                            data_name_A="standard data", data_name_B="stylized data",
                            data_name_C="combined data", color_A=col_clean,
                            color_B = col_stylized, color_C = col_combined):

    fig = plt.figure(figsize = (15.0, 20.0))
    plt.subplots_adjust(wspace = 0.3, hspace = 0.5)

    for i, distortion in enumerate(distortions[0:15]): # only plot first 15 distortions, not the validation distortions
        ax = plt.subplot(5,3,i+1)

        plt.plot(list(range(6)), data_A[i,:] * 100, 's-', zorder=2, color=color_A,
                 label=data_name_A, markersize=markersize, linewidth=linewidth)
        plt.plot(list(range(6)), data_B[i,:] * 100, '^-', zorder=1, color=color_B,
                 label=data_name_B, markersize=markersize, linewidth=linewidth)
        plt.plot(list(range(6)), data_C[i,:] * 100, 'o-', zorder=3, color=color_C,
                 label=data_name_C, markersize=markersize, linewidth=linewidth)


        plt.title(distortion, fontsize=fontsize)    
        if i >= 12: # plot xlabel only in last row
            plt.xlabel('corruption severity', fontsize=fontsize)
        if i % 3 == 0:
            plt.ylabel(metric_name, fontsize=fontsize)
        plt.axis([axis_start, axis_end, ylim[0], ylim[1]])
        if i == 0:
            legend = plt.legend(loc=legend_loc, fontsize=legend_fontsize, frameon=True, edgecolor="black")
            legend.get_frame().set_linewidth(1.0)

        ax.tick_params(axis='both', which='major', labelsize=labelsize)


    sns.despine(trim=True, offset=5)

    plt.show()

    fig.savefig(pjoin(figures_dir, plot_name), bbox_inches='tight')
    

### Pascal VOC

In [ ]:
distortions = get_distortions_from_file(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_voc0712_results.pkl"))
voc_results = get_results(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_voc0712_results.pkl"), dataset='voc')
svoc_results = get_results(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_svoc0712_results.pkl"), dataset='voc')
voc_svoc_results = get_results(pjoin(result_dir, "pascal_voc/faster_rcnn_r50_fpn_1x_vocsvoc0712_results.pkl"), dataset='voc')

In [ ]:
def set_fontsize_helper(ax, fontsize):
    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fontsize)

In [ ]:
# Pascal VOC combined=averaged plot

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)
plt.plot(list(range(6)), np.mean(voc_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='standard data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(svoc_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(voc_svoc_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='combined data', markersize=markersize,
        linewidth=linewidth)

plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP50 in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 85])
ax.tick_params(axis='both', which='major', labelsize=labelsize)
legend = plt.legend(loc=3, fontsize=legend_fontsize, frameon=True, edgecolor="black")
legend.get_frame().set_linewidth(1.0)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, 'pascal_corruption_overall.pdf'), bbox_inches='tight')

In [ ]:
plot_individual_results(data_A = voc_results,
                        data_B=svoc_results,
                        data_C=voc_svoc_results,
                        plot_name="pascal_corruption_individual.pdf",
                        metric_name="mAP50 in %", ylim=[0, 85], legend_loc=3)

In [ ]:
import csv
from imagecorruptions import get_corruption_names

corruption_names = get_corruption_names('all')

clean_performance = voc_results[0,0,0]
mean_voc_results = np.mean(voc_results[:15,1:,0], axis=1)
mean_voc_svoc_results = np.mean(voc_svoc_results[:15, 1:,0], axis=1)

rpc_voc_clean = mean_voc_results / clean_performance
rpc_voc_combined = mean_voc_svoc_results / clean_performance

# load the rmse values calculated with the calc_rmse.py script
rmse_voc = np.zeros((19, 5))
with open('../raw-data/pascal_voc/voc_rmse.csv') as f:
    reader = csv.DictReader(f)
    for rownum, row in enumerate(reader):
        corruption_number = corruption_names.index(row['corruption'])
        rmse_voc[corruption_number, int(row['severity']) - 1] = row['RMSE']
        
mean_rmse_voc = np.mean(rmse_voc, axis=1)

num_noise = 3
num_blur = 4
num_weather = 4
num_digital = 4
noise = np.array([145,105,70]) / 255.
blur = np.array([50, 110, 30]) / 255.
weather = np.array([80, 170, 200]) / 255.
digital = np.array([175, 110, 150]) / 255.

colors = [noise for _ in range(num_noise)] + [blur for _ in range(num_blur)] + [weather for _ in range(num_weather)] + [digital for _ in range(num_digital)]

In [ ]:
fig = plt.figure(figsize = (10.0, 4.0))

ax = plt.subplot(1,1,1)

rmse_fontsize=14
rmse_linewidth=2
rmse_markersize=8
rmse_capsize=4

plt.axis([2, 12, 0, 100])
ax.tick_params(axis='both', which='major', labelsize=rmse_fontsize)

plt.xlabel('RMSE', fontsize=rmse_fontsize)
plt.ylabel('rPC[%]', fontsize=rmse_fontsize)
x = mean_rmse_voc
y = rpc_voc_clean * 100
ywhiskers = (rpc_voc_combined - rpc_voc_clean) * 100
xe = x[:num_noise]
ye = y[:num_noise]
ywe = ywhiskers[:num_noise]
lolims = np.ones(num_noise, dtype=np.bool)
uplims = np.zeros(num_noise, dtype=np.bool)
plt.errorbar(xe, ye, yerr=ywe, lolims=lolims, uplims=uplims, linestyle="None", ecolor=noise, marker='o', mfc=noise, linewidth=rmse_linewidth, markersize=rmse_markersize, capsize=rmse_capsize)

xe = x[num_noise:num_noise+num_blur]
ye = y[num_noise:num_noise+num_blur]
ywe = ywhiskers[num_noise:num_noise+num_blur]
lolims = np.ones(num_blur, dtype=np.bool)
uplims = np.zeros(num_blur, dtype=np.bool)
plt.errorbar(xe, ye, yerr=ywe, lolims=lolims, uplims=uplims, linestyle="None", ecolor=blur, marker='s', mfc=blur, linewidth=rmse_linewidth, markersize=rmse_markersize, capsize=rmse_capsize)

xe = x[num_noise+num_blur:num_noise+num_blur+num_weather]
ye = y[num_noise+num_blur:num_noise+num_blur+num_weather]
ywe = ywhiskers[num_noise+num_blur:num_noise+num_blur+num_weather]
lolims = np.ones(num_weather, dtype=np.bool)
uplims = np.zeros(num_weather, dtype=np.bool)
plt.errorbar(xe, ye, yerr=ywe, lolims=lolims, uplims=uplims, linestyle="None", ecolor=weather, marker='D', mfc=weather, linewidth=rmse_linewidth, markersize=rmse_markersize, capsize=rmse_capsize)

xe = x[num_noise+num_blur+num_weather:num_noise+num_blur+num_weather+num_digital]
ye = y[num_noise+num_blur+num_weather:num_noise+num_blur+num_weather+num_digital]
ywe = ywhiskers[num_noise+num_blur+num_weather:num_noise+num_blur+num_weather+num_digital]
lolims = np.ones(num_digital, dtype=np.bool)
uplims = np.zeros(num_digital, dtype=np.bool)
plt.errorbar(xe, ye, yerr=ywe, lolims=lolims, uplims=uplims, linestyle="None", ecolor=digital, marker='X', mfc=digital, linewidth=rmse_linewidth, markersize=rmse_markersize, capsize=rmse_capsize)

text_offsets = [
    (4, -4), # gaussian noise 0
    (-4, -4), # shot noise 1
    (4, -4), # impulse noise 2
    (-4, -4), # defocus blur 3
    (4, -4), # glass blur 4
    (4, -4), # motion blur 5
    (4, -4), # zoom blur 6
    (4, -4), # snow 7
    (-4, 0), # frost 8
    (4, -4), # fog 9
    (-4, -4), # brightness 10
    (-4, 0), # contrast 11
    (4, -4), # elastic transform 12
    (-4, -4), # pixelate 13
    (4, 0), # jpeg_compression 14
]

annotations = [
    'gaussian',
    'shot',
    'impulse',
    'defocus',
    'glass',
    'motion',
    'zoom',
    'snow',
    'frost',
    'fog',
    'brightness',
    'contrast',
    'elastic',
    'pixelate',
    'jpeg'
]

ha = [
    'left',
    'right',
    'left',
    'right',
    'left',
    'left',
    'left',
    'left',
    'right',
    'left',
    'right',
    'right',
    'left',
    'right',
    'left',
]
ax = plt.gca()
for c in range(15):
    ax.annotate(annotations[c], (x[c], y[c]), xytext=text_offsets[c], textcoords='offset pixels', fontsize=rmse_fontsize, ha = ha[c], color=colors[c])

legend = plt.legend(['noise', 'blur', 'weather', 'digital'], loc='lower left', fontsize=rmse_fontsize, frameon=True, edgecolor="black")
for text, color in zip(legend.get_texts(), [noise, blur, weather, digital]):
    plt.setp(text, color=color)
legend.get_frame().set_linewidth(1.0)
    
sns.despine(trim=True, offset=5)

plt.show()
fig.savefig(pjoin(figures_dir, "rpc_vs_rmse.pdf"), bbox_inches='tight')

### MS COCO

In [ ]:
distortions = get_distortions_from_file(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_coco_results.pkl"))
coco_results = get_results(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_coco_results.pkl"), metric='AP')
scoco_results = get_results(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_scoco_results.pkl"), metric='AP')
coco_scoco_results = get_results(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_cocoscoco_results.pkl"), metric='AP')

In [ ]:
# MS COCO combined results

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)

plt.plot(list(range(6)), np.mean(coco_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='coco', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(scoco_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized coco', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(coco_scoco_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='combined', markersize=markersize,
        linewidth=linewidth)


plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 40])
ax.tick_params(axis='both', which='major', labelsize=labelsize)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, "coco_corruption_overall.pdf"), bbox_inches='tight')

In [ ]:
plot_individual_results(data_A = coco_results,
                        data_B=scoco_results,
                        data_C=coco_scoco_results,
                        plot_name="coco_corruption_individual.pdf",
                        metric_name="mAP in %", ylim=[0, 40])

### MS COCO Benchmark

In [ ]:
metric = 'AP'
distortions = get_distortions_from_file(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_results.pkl"))

coco_frcnn_r50_results = get_results(pjoin(result_dir, "coco/faster_rcnn_r50_fpn_1x_results.pkl"), metric=metric)
coco_frcnn_r101_results = get_results(pjoin(result_dir, "coco/faster_rcnn_r101_fpn_1x_results.pkl"), metric=metric)
coco_frcnn_x101_32x4d_results = get_results(pjoin(result_dir, "coco/faster_rcnn_x101_32x4d_fpn_1x_results.pkl"), metric=metric)
coco_frcnn_x101_64x4d_results = get_results(pjoin(result_dir, "coco/faster_rcnn_x101_64x4d_fpn_1x_results.pkl"), metric=metric)
    
coco_mrcnn_r50_results = get_results(pjoin(result_dir, "coco/mask_rcnn_r50_fpn_1x_results.pkl"), metric=metric)
coco_crcnn_r50_results = get_results(pjoin(result_dir, "coco/cascade_rcnn_r50_fpn_1x_results.pkl"), metric=metric)
coco_cmrcnn_r50_results = get_results(pjoin(result_dir, "coco/cascade_mask_rcnn_r50_fpn_1x_results.pkl"), metric=metric)
coco_retina_r50_results = get_results(pjoin(result_dir, "coco/retinanet_r50_fpn_1x_results.pkl"), metric=metric)
    
coco_frcnn_dcn_r50_results = get_results(pjoin(result_dir, "coco/faster_rcnn_dconv_c3-c5_r50_fpn_1x_results.pkl"), metric=metric)
coco_frcnn_dcn_x101_32x4d_results = get_results(pjoin(result_dir, "coco/faster_rcnn_dconv_c3-c5_x101_32x4d_fpn_1x_results.pkl"), metric=metric)
coco_mrcnn_dcn_r50_results = get_results(pjoin(result_dir, "coco/mask_rcnn_dconv_c3-c5_r50_fpn_1x_results.pkl"), metric=metric)
coco_htc_dcn_x101_64x4d_results = get_results(pjoin(result_dir, "coco/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_results.pkl"), metric=metric)

coco_mrcnn_r50_results_segm = get_results(pjoin(result_dir, "coco/mask_rcnn_r50_fpn_1x_results.pkl"), task='segm', metric=metric)
coco_cmrcnn_r50_results_segm = get_results(pjoin(result_dir, "coco/cascade_mask_rcnn_r50_fpn_1x_results.pkl"), task='segm', metric=metric)
coco_mrcnn_dcn_r50_results_segm = get_results(pjoin(result_dir, "coco/mask_rcnn_dconv_c3-c5_r50_fpn_1x_results.pkl"), task='segm', metric=metric)
coco_htc_dcn_x101_64x4d_results_segm = get_results(pjoin(result_dir, "coco/htc_dconv_c3-c5_mstrain_400_1400_x101_64x4d_fpn_20e_results.pkl"), task='segm', metric=metric)

In [ ]:
result_list = [coco_frcnn_r50_results, 
               coco_frcnn_r101_results, 
                coco_frcnn_x101_32x4d_results, 
                coco_frcnn_x101_64x4d_results,
                coco_frcnn_dcn_r50_results,
                coco_frcnn_dcn_x101_32x4d_results,
                coco_mrcnn_dcn_r50_results,
                coco_htc_dcn_x101_64x4d_results,
                coco_mrcnn_r50_results, 
                coco_crcnn_r50_results, 
                coco_cmrcnn_r50_results, 
                coco_retina_r50_results, 
                coco_mrcnn_r50_results_segm, 
                coco_cmrcnn_r50_results_segm, 
                coco_mrcnn_dcn_r50_results_segm,
                coco_htc_dcn_x101_64x4d_results_segm]

model_names = ['frcnn_r50', 
                'frcnn_r101', 
                'frcnn_x101_32x4d', 
                'frcnn_x101_64x4d',
                'frcnn_dcn_r50',
                'frcnn_dcn_x101_32x4d',
                'mrcnn_dcn_r50',
                'htc_dcn_x101_64x4d',
                'mrcnn_r50', 
                'crcnn_r50', 
                'cmrcnn_r50', 
                'retinanet_r50', 
                'mrcnn_r50_segm', 
                'cmrcnn_r50_segm',
                'mrcnn_dcn_r50_segm',
                'htc_dcn_x101_64x4d_segm']

colors = ['black','dark red', 'red', 'orange', 'dandelion']

In [ ]:
for i, results in enumerate(result_list):
    print('{}: clean:{:.1f} corr.:{:.1f} rel:{:.1f}%'.format(model_names[i], 
                                                           results[0,0,0] * 100, 
                                                           np.mean(results[:15,1:,0]) * 100, 
                                                           np.mean(results[:15,1:,0])/results[0,0,0] * 100))
    
    print('')

In [ ]:
# plot results for different backbones

# use different colour scheme to avoid confusion
col_resnet50 = col_clean
col_resnet101 = tuple(x/255.0 for x in (175, 110, 150))
col_resnext101 = tuple(x/255.0 for x in (210, 150, 0))

plot_individual_results(data_A = coco_frcnn_r50_results,
                        data_B=coco_frcnn_r101_results,
                        data_C=coco_frcnn_x101_64x4d_results,
                        color_A=col_resnet50,
                        color_B=col_resnet101,
                        color_C=col_resnext101,
                        data_name_A="ResNet-50",
                        data_name_B="ResNet-101",
                        data_name_C="ResNeXt-101",
                        plot_name="coco_corruption_backbones_individual.pdf",
                        metric_name="mAP in %", ylim=[-1, 45])

## Citscapes

In [ ]:
distortions = get_distortions_from_file(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_city_results.pkl"))

city_results = get_results(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_city_results.pkl"), metric='AP')
scity_results = get_results(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_scity_results.pkl"), metric='AP')
city_scity_results = get_results(pjoin(result_dir, "cityscapes/faster_rcnn_r50_fpn_1x_cityscity_results.pkl"), metric='AP')

In [ ]:
# Cityscapes combined results

fig = plt.figure(figsize = (combined_plot_width, combined_plot_height))

ax = plt.subplot(1,1,1)

plt.plot(list(range(6)), np.mean(city_results[:15,:], axis=0) * 100,
         's-', zorder=2, color=col_clean, label='standard data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(scity_results[:15,:], axis=0) * 100,
         '^-', zorder=1, color=col_stylized, label='stylized data', markersize=markersize,
        linewidth=linewidth)
plt.plot(list(range(6)), np.mean(city_scity_results[:15,:], axis=0) * 100,
         'o-', zorder=3, color=col_combined, label='including stylized data', markersize=markersize,
        linewidth=linewidth)


plt.xlabel('corruption severity', fontsize=fontsize)
plt.ylabel('mAP in %', fontsize=fontsize)
plt.axis([axis_start, axis_end, 0, 40])
ax.tick_params(axis='both', which='major', labelsize=labelsize)

sns.despine(trim=True, offset=5)

plt.show()

fig.savefig(pjoin(figures_dir, "cityscapes_corruption_overall.pdf"), bbox_inches='tight')

In [ ]:
plot_individual_results(data_A = city_results,
                        data_B = scity_results,
                        data_C = city_scity_results,
                        plot_name="cityscapes_corruption_individual.pdf",
                        metric_name="mAP in %", ylim=[-1, 40])